In [2]:
import pandas as pd
import numpy as np

# Make TPM matrix

In [42]:
experiments = pd.read_csv("../data/intermediate/experiment_setup.txt",sep="\t")

experiments["kallisto_path"] = "../data/intermediate/kallisto_quant/"+experiments["sample"]+"/abundance.tsv"

def load_tpms(exp_path, exp_name):
    abundances = pd.read_csv(exp_path,sep="\t",index_col=0)
    
    tpms = abundances["tpm"]
    tpms = tpms.rename(exp_name)
    tpms = tpms.astype(np.float64)
    
    return tpms

In [43]:
exps = zip(experiments["kallisto_path"],experiments["sample"])

merged_tpms = [load_tpms(x[0],x[1]) for x in exps]

merged_tpms = pd.concat(merged_tpms,axis=1)

merged_tpms.to_csv("../data/processed/tpm_matrix.txt",sep="\t")

In [62]:
suppa_tpms = merged_tpms.copy(deep=True)

suppa_tpms.index = suppa_tpms.index.map(lambda x: x.split(".")[0])

suppa_tpms.to_csv("../data/processed/tpm_matrix_suppa.txt",sep="\t",index_label=False)

# Transcripts to genes

In [44]:
t2g = pd.read_csv("../data/raw/kallisto_homo_sapiens/transcripts_to_genes.txt",
                  sep="\t",
                  names=["transcript_id","gene_id","gene_name"]
                 )
t2g["format_transcript_id"] = t2g["gene_name"] + "_" + t2g["transcript_id"]
t2g["format_gene_id"] = t2g["gene_name"] + "_" + t2g["gene_id"]

transcript_map = dict(zip(t2g["transcript_id"],t2g["format_transcript_id"]))

In [45]:
named_tpms = merged_tpms.copy(deep=True)

named_tpms.index = named_tpms.index.map(lambda x: transcript_map.get(x,"unnamed_"+x))
named_tpms = named_tpms.T